In [1]:
import pandas as pd

ab01 = pd.read_csv('./data/apollo/ab01.csv')
ab02 = pd.read_csv('./data/apollo/ab02.csv')
em01 = pd.read_csv('./data/apollo/em01.csv')
ab01.head()

In [2]:
ab02.head()

,co_code,fin_class,ref_date,rep_code,item_code,ratio,unkown
0,43,K,20111231,19,1010,8.888889e+11,NaN
1,43,K,20111231,19,1020,8.888889e+11,NaN
2,43,K,20111231,19,1030,8.888889e+11,NaN
3,43,K,20111231,19,1040,8.888889e+11,NaN
4,43,K,20111231,19,1050,8.888889e+11,NaN


In [2]:
ab01 = ab01[['co_code','ref_date','rep_code','item_code','amt']]
ab02 = ab02[['co_code','ref_date','rep_code','item_code','ratio']]

In [3]:
gaap = pd.read_excel('./NICE_2023_레이아웃_코드/재무계정코드(GAAP 외부)_20190213.xlsx')
gaap.head()

,보고서코드,항목코드,항목명,항목명(영문)
0,10,5000,종업원수,NO. OF EMPLOYEES
1,10,0,일 반 사 항,NaN
2,10,2000,감사의견,NaN
3,10,3000,결산기,NaN
4,11,2000,유동자산(계),TOTAL CURRENT ASSETS


In [4]:
gaap = gaap[['보고서코드','항목코드','항목명']]
gaap.rename(columns={'보고서코드':'rep_code','항목코드':'item_code'}, inplace=True)

In [6]:
gaap

,rep_code,item_code,항목명
0,10,5000,종업원수
1,10,0,일 반 사 항
2,10,2000,감사의견
3,10,3000,결산기
4,11,2000,유동자산(계)
...,...,...,...
3612,49,9995,성장성(%)
3613,78,0,주당순이익
3614,80,0,주당순자산
3615,88,0,배당수익율


# ab01, ab02 전처리 순서

1. GAAP 매핑
2. 이상치 제거 ex) 비율 데이터 100,000,000,000 이상 삭제 등
3. 특수문자, 연속공백, 공백 제거
4. co_code기준 em01의 br_num, fin_year, cl_code와 매핑
5. fin_year와 ref_date 같은 달에 해당하는 row만 남기기
5. cl_code N인 row만 남기기
6. 한 기업 한 년도를 하나의 row로 만들기
7. 업체코드, 기준일자, 항목명 중복시 제거 (첫번째 row 남기기)
8. 선정 변수만 남기기 (30개)

-----------------ab01, ab02 merge 전--------------------

9. 내년매출액증가율 (LABEL) 만들기

In [5]:
import re

# 특수문자, 연속된 공백, 모든 공백 제거 함수
def clean_string(s):
    # 특수문자 제거
    s = re.sub(r'[^\w\s]', '', s)
    # 연속된 공백 제거 (공백을 하나로 줄이기)
    s = re.sub(r'\s+', ' ', s)
    # 모든 공백 제거
    s = s.replace(' ', '')
    return s

gaap['항목명'] = gaap['항목명'].apply(clean_string)

In [6]:
ab01 = pd.merge(ab01, gaap, how='left', on=['rep_code','item_code'])
ab02 = pd.merge(ab02, gaap, how='left', on=['rep_code','item_code'])
ab01.head()

,co_code,ref_date,rep_code,item_code,amt,항목명
0,43,20111231,10,5000,10.0,종업원수
1,43,20111231,11,1100,650400.0,현금및현금등가물
2,43,20111231,11,1110,105666.0,현금
3,43,20111231,11,1130,544735.0,예금
4,43,20111231,11,1134,544735.0,기타예금


In [9]:
ab02.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463866509 entries, 0 to 463866508
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   co_code    object 
 1   ref_date   int64  
 2   rep_code   object 
 3   item_code  int64  
 4   ratio      float64
 5   항목명        object 
dtypes: float64(1), int64(2), object(3)
memory usage: 20.7+ GB


In [7]:
print(len(ab02))
ab02 = ab02[ab02['ratio']<100000000000]
print(len(ab02))

463866509
336774608


In [8]:
ab01 = ab01[['co_code','ref_date','amt','항목명']]
ab02 = ab02[['co_code','ref_date','ratio','항목명']]

In [12]:
ab01

,co_code,ref_date,amt,항목명
0,43,20111231,10.0,종업원수
1,43,20111231,650400.0,현금및현금등가물
2,43,20111231,105666.0,현금
3,43,20111231,544735.0,예금
4,43,20111231,544735.0,기타예금
...,...,...,...,...
646554747,Z99966,20191231,194394.0,유상증자
646554748,Z99966,20191231,194394.0,외부자금조달
646554749,Z99966,20191231,262316.0,현금및현금등가물의순증
646554750,Z99966,20191231,64982.0,기초현금및현금등가물


In [9]:
ab_cols = list(pd.read_pickle('./data/apollo/풀데이터/full_2020_train_node.pkl').columns)
ab_cols

['감가상각율',
 '금융비용총부채',
 '금융비용총비용비율',
 '당좌비율',
 '매출원가율',
 '매출채권회전율',
 '배당성향',
 '부채비율',
 '비유동자산비율',
 '비유동자산장기적합율',
 '비유동자산회전율',
 '수지비율',
 '영업이익증가율',
 '유형자산증가율',
 '유형자산회전율',
 '자기자본증가율',
 '자본금회전율',
 '재고자산회전율1',
 '총자본회전율',
 '총자산증가율',
 '인건비',
 '매출액',
 '매출액증가율',
 'label']

In [14]:
len(ab_cols)

24

In [10]:
ab01 = ab01[ab01['항목명'].isin(ab_cols)]
ab02 = ab02[ab02['항목명'].isin(ab_cols)]

In [16]:
ab01

,co_code,ref_date,amt,항목명
99,43,20111231,3997859.0,매출액
120,43,20111231,328520.0,인건비
316,43,20121231,5853130.0,매출액
337,43,20121231,304277.0,인건비
570,43,20131231,5342488.0,매출액
...,...,...,...,...
646554515,Z99966,20171231,234902.0,인건비
646554587,Z99966,20181231,511246.0,매출액
646554597,Z99966,20181231,234671.0,인건비
646554686,Z99966,20191231,1174292.0,매출액


In [11]:
# 하나의 row로 만들기

pivot_ab01 = ab01.pivot_table(index=['co_code', 'ref_date'], 
                          columns='항목명', 
                          values='amt',
                          aggfunc='first')

# 열 이름을 다시 정리 (MultiIndex 제거)
pivot_ab01 = pivot_ab01.reset_index()

pivot_ab01

항목명,co_code,ref_date,매출액,인건비
0,43,20111231,3997859.0,328520.0
1,43,20121231,5853130.0,304277.0
2,43,20131231,5342488.0,298645.0
3,43,20141231,5152680.0,360815.0
4,43,20151231,4422247.0,517310.0
...,...,...,...,...
2548788,Z99879,20091231,180000.0,25613.0
2548789,Z99879,20101231,165000.0,24061.0
2548790,Z99966,20171231,449058.0,234902.0
2548791,Z99966,20181231,511246.0,234671.0


In [12]:
# 하나의 row로 만들기

pivot_ab02 = ab02.pivot_table(index=['co_code', 'ref_date'], 
                          columns='항목명', 
                          values='ratio',
                          aggfunc='first')

# 열 이름을 다시 정리 (MultiIndex 제거)
pivot_ab02 = pivot_ab02.reset_index()

pivot_ab02

항목명,co_code,ref_date,감가상각율,금융비용총부채,금융비용총비용비율,당좌비율,매출액증가율,매출원가율,매출채권회전율,배당성향,...,비유동자산회전율,수지비율,영업이익증가율,유형자산증가율,유형자산회전율,자기자본증가율,자본금회전율,재고자산회전율1,총자본회전율,총자산증가율
0,43,20111231,17.62,0.00,0.00,602.79,NaN,85.37,3.46,0.0,...,3.53,98.91,NaN,NaN,3.59,NaN,10.41,17.13,1.14,NaN
1,43,20121231,17.94,1.21,0.15,738.66,46.41,79.03,4.55,0.0,...,4.56,88.42,4826.40,26.90,4.63,22.05,15.24,16.83,1.45,31.06
2,43,20131231,26.10,1.42,0.24,1074.82,-8.72,75.58,4.73,0.0,...,3.81,85.94,7.20,-4.12,3.86,17.40,13.91,14.59,1.10,11.88
3,43,20141231,26.25,1.23,0.16,1294.61,-3.55,81.47,5.48,0.0,...,3.92,92.91,-64.85,-8.99,3.98,5.70,13.42,15.44,1.00,1.28
4,43,20151231,27.14,0.36,0.04,1022.13,-14.18,85.33,4.27,NaN,...,3.78,103.24,NaN,-14.04,3.86,-4.34,11.52,8.35,0.88,-7.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695380,Z99879,20091231,19.37,5.54,72.75,252.73,-47.06,0.00,NaN,NaN,...,0.07,142.01,-57.68,-0.13,0.07,-27.66,1.20,NaN,0.05,4.26
1695381,Z99879,20101231,13.19,6.52,90.18,184.25,-8.33,0.00,NaN,NaN,...,0.06,205.01,19.44,-0.07,0.06,NaN,2.63,NaN,0.05,-24.60
1695382,Z99966,20171231,NaN,0.00,0.00,12698.58,NaN,23.38,NaN,0.0,...,5.16,93.76,NaN,NaN,35.54,NaN,1.59,NaN,1.58,NaN
1695383,Z99966,20181231,0.00,0.00,0.00,1598.93,13.85,17.78,NaN,0.0,...,6.30,75.37,112.37,-91.01,74.24,-8.34,1.89,NaN,1.84,-4.53


In [14]:
# em01 = em01[['co_code','br_num','cl_code','fin_mon']]

em01 = em01[['co_code','br_num','cl_code','fin_mon']]
pivot_ab01 = pd.merge(pivot_ab01, em01, how='left', on='co_code')
pivot_ab02 = pd.merge(pivot_ab02, em01, how='left', on='co_code')

In [16]:
pivot_ab01

,co_code,ref_date,매출액,인건비,br_num,cl_code,fin_mon
0,43,20111231,3997859.0,328520.0,NaN,NaN,NaN
1,43,20121231,5853130.0,304277.0,NaN,NaN,NaN
2,43,20131231,5342488.0,298645.0,NaN,NaN,NaN
3,43,20141231,5152680.0,360815.0,NaN,NaN,NaN
4,43,20151231,4422247.0,517310.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2548788,Z99879,20091231,180000.0,25613.0,5.021639e+09,X,12
2548789,Z99879,20101231,165000.0,24061.0,5.021639e+09,X,12
2548790,Z99966,20171231,449058.0,234902.0,3.170137e+09,X,12
2548791,Z99966,20181231,511246.0,234671.0,3.170137e+09,X,12


In [21]:
pivot_ab01.isnull().sum()/ len(pivot_ab01)

co_code     0.000000
ref_date    0.000000
매출액         0.011606
인건비         0.078392
br_num      0.126869
cl_code     0.126869
fin_mon     0.126869
dtype: float64

In [22]:
pivot_ab02.isnull().sum()/ len(pivot_ab02)

co_code       0.000000
ref_date      0.000000
감가상각율         0.113355
금융비용총부채       0.006760
금융비용총비용비율     0.001344
당좌비율          0.014003
매출액증가율        0.188652
매출원가율         0.016640
매출채권회전율       0.194408
배당성향          0.150057
부채비율          0.060392
비유동자산비율       0.060395
비유동자산장기적합율    0.037536
비유동자산회전율      0.012234
수지비율          0.004223
영업이익증가율       0.370538
유형자산증가율       0.201922
유형자산회전율       0.033924
자기자본증가율       0.233966
자본금회전율        0.009978
재고자산회전율1      0.325699
총자본회전율        0.000235
총자산증가율        0.175433
br_num        0.133714
cl_code       0.133714
fin_mon       0.133714
dtype: float64

In [17]:
# br_num 결측치 기업 삭제
print(len(pivot_ab01), len(pivot_ab02))
pivot_ab01.dropna(subset='br_num',inplace=True)
pivot_ab02.dropna(subset='br_num',inplace=True)
print(len(pivot_ab01), len(pivot_ab02))

2548793 1695385
2225431 1468688


In [24]:
pivot_ab01.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2225431 entries, 90454 to 2548792
Data columns (total 7 columns):
 #   Column    Dtype  
---  ------    -----  
 0   co_code   object 
 1   ref_date  int64  
 2   매출액       float64
 3   인건비       float64
 4   br_num    float64
 5   cl_code   object 
 6   fin_mon   object 
dtypes: float64(3), int64(1), object(3)
memory usage: 135.8+ MB


In [25]:
import numpy as np

pivot_ab01['fin_mon'].replace('  ', np.NaN, inplace=True)
pivot_ab02['fin_mon'].replace('  ', np.NaN, inplace=True)

pivot_ab01.dropna(subset='fin_mon',inplace=True)
pivot_ab02.dropna(subset='fin_mon',inplace=True)

pivot_ab01['fin_mon'] = pivot_ab01['fin_mon'].astype(int)
pivot_ab02['fin_mon'] = pivot_ab02['fin_mon'].astype(int)

pivot_ab01['ref_date'] = pivot_ab01['ref_date'].astype('str')
pivot_ab01['month'] = pivot_ab01['ref_date'].str.slice(start=4, stop=6)
pivot_ab02['ref_date'] = pivot_ab02['ref_date'].astype('str')
pivot_ab02['month'] = pivot_ab02['ref_date'].str.slice(start=4, stop=6)

ab01 = pivot_ab01[pivot_ab01['month'].astype(int)==pivot_ab01['fin_mon']]
ab02 = pivot_ab02[pivot_ab02['month'].astype(int)==pivot_ab02['fin_mon']]
print(len(ab01), len(ab02))

2212815 1460486


In [26]:
ab01 = ab01[ab01['cl_code']=='N']
ab02 = ab02[ab02['cl_code']=='N']
print(len(ab01), len(ab02))

1486102 894650


In [27]:
ab01 = pivot_ab01
ab02 = pivot_ab02

In [28]:
ab01['ref_date'] = ab01['ref_date'].astype('str')
ab02['ref_date'] = ab02['ref_date'].astype('str')

In [20]:
ab01.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2225431 entries, 90454 to 2548792
Data columns (total 7 columns):
 #   Column    Dtype  
---  ------    -----  
 0   co_code   object 
 1   ref_date  int64  
 2   매출액       float64
 3   인건비       float64
 4   br_num    int64  
 5   cl_code   object 
 6   fin_mon   object 
dtypes: float64(2), int64(2), object(3)
memory usage: 135.8+ MB


In [29]:
ab01['br_num'] = ab01['br_num'].astype('int64')
ab02['br_num'] = ab02['br_num'].astype('int64')

ab01['br_num_year'] = ab01['br_num'].astype(str) + '_' + ab01['ref_date'].str.slice(start=2, stop=4)
ab02['br_num_year'] = ab02['br_num'].astype(str) + '_' + ab02['ref_date'].str.slice(start=2, stop=4)

ab01.head()

,co_code,ref_date,매출액,인건비,br_num,cl_code,fin_mon,br_num_year
90454,87392,20141231,2438243.0,337440.0,1258141629,N,12,1258141629_14
90455,87392,20151231,2828597.0,743880.0,1258141629,N,12,1258141629_15
90456,87392,20161231,4055821.0,306915.0,1258141629,N,12,1258141629_16
90457,87392,20171231,3201388.0,576136.0,1258141629,N,12,1258141629_17
90458,87392,20181231,2476843.0,626114.0,1258141629,N,12,1258141629_18


In [23]:
ab02.columns

Index(['co_code', 'ref_date', '감가상각율', '금융비용총부채', '금융비용총비용비율', '당좌비율',
       '매출액증가율', '매출원가율', '매출채권회전율', '배당성향', '부채비율', '비유동자산비율', '비유동자산장기적합율',
       '비유동자산회전율', '수지비율', '영업이익증가율', '유형자산증가율', '유형자산회전율', '자기자본증가율',
       '자본금회전율', '재고자산회전율1', '총자본회전율', '총자산증가율', 'br_num', 'cl_code',
       'fin_mon', 'br_num_year'],
      dtype='object')

In [30]:
ab01 = ab01[['br_num_year', '매출액','인건비']]
ab02 = ab02[['br_num_year', '감가상각율', '금융비용총부채', '금융비용총비용비율', '당좌비율',
       '매출액증가율', '매출원가율', '매출채권회전율', '배당성향', '부채비율', '비유동자산비율', '비유동자산장기적합율',
       '비유동자산회전율', '수지비율', '영업이익증가율', '유형자산증가율', '유형자산회전율', '자기자본증가율',
       '자본금회전율', '재고자산회전율1', '총자본회전율', '총자산증가율']]

print(len(ab01.columns))
print(len(ab02.columns))

3
22


In [31]:
ab = pd.merge(ab01, ab02, how='inner', on='br_num_year')
ab

,br_num_year,매출액,인건비,감가상각율,금융비용총부채,금융비용총비용비율,당좌비율,매출액증가율,매출원가율,매출채권회전율,...,비유동자산회전율,수지비율,영업이익증가율,유형자산증가율,유형자산회전율,자기자본증가율,자본금회전율,재고자산회전율1,총자본회전율,총자산증가율
0,4108170342_22,233089621.0,3197197.0,NaN,NaN,NaN,NaN,NaN,NaN,4.59,...,2.84,NaN,NaN,NaN,7.01,NaN,2330.90,13.45,0.38,NaN
1,2100777518_17,509579.0,39408.0,18.89,0.00,0.00,60.75,NaN,69.49,NaN,...,10.48,92.27,NaN,NaN,16.12,NaN,5.43,9.72,4.54,NaN
2,2100777518_18,575792.0,38639.0,25.82,0.00,0.00,52.73,12.99,71.93,226.07,...,12.92,95.41,29.06,-25.82,20.92,-22.55,6.91,11.52,5.30,-6.40
3,2100777518_19,663535.0,48480.0,33.52,0.00,0.00,19.24,15.24,69.43,173.81,...,18.15,94.89,25.02,-33.53,34.01,-78.60,15.04,26.40,9.14,-61.83
4,2100777518_20,597452.0,61812.0,44.40,4.00,0.84,5.58,-9.96,65.24,215.18,...,20.32,94.90,5.23,-41.29,48.32,NaN,76.83,75.23,13.92,14.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429161,5021639400_09,180000.0,25613.0,19.37,5.54,72.75,252.73,-47.06,0.00,NaN,...,0.07,142.01,-57.68,-0.13,0.07,-27.66,1.20,NaN,0.05,4.26
1429162,5021639400_10,165000.0,24061.0,13.19,6.52,90.18,184.25,-8.33,0.00,NaN,...,0.06,205.01,19.44,-0.07,0.06,NaN,2.63,NaN,0.05,-24.60
1429163,3170137061_17,449058.0,234902.0,NaN,0.00,0.00,12698.58,NaN,23.38,NaN,...,5.16,93.76,NaN,NaN,35.54,NaN,1.59,NaN,1.58,NaN
1429164,3170137061_18,511246.0,234671.0,0.00,0.00,0.00,1598.93,13.85,17.78,NaN,...,6.30,75.37,112.37,-91.01,74.24,-8.34,1.89,NaN,1.84,-4.53


In [32]:
ab['year'] = ab['br_num_year'].str.slice(start=11, stop=13)
ab['year'].value_counts()

year
17    133228
16    129752
18    128055
19    123199
15    122739
14    114570
13    105617
20    104224
12     91230
21     88334
11     71904
22     59377
10     58467
08     50385
09     48085
Name: count, dtype: int64

In [40]:
# bg = pd.read_csv('./data/apollo/bg03.csv')

In [41]:
# bg['br_num'] = bg['br_num'].astype('Int64')
# bg['cp_br_num'] = bg['cp_br_num'].astype('Int64')
# bg.dropna(subset='br_num', inplace=True)
# bg.dropna(subset='cp_br_num', inplace=True)


In [42]:
# bg['br_num'] = bg['br_num'].astype(str)
# bg['txn_end'] = bg['txn_end'].astype(str)
# bg['year'] = bg['txn_end'].str.slice(start=2, stop=4)
# bg['year'].value_counts()

year
21    23430656
20    23357869
19    23168499
18    21114381
22    20391976
Name: count, dtype: int64

In [38]:
# # bg['br_num'] = bg['br_num'].astype('int16')
# bg['br_num'] = bg['br_num'].astype(str)
# bg['txn_end'] = bg['txn_end'].astype(str)
# bg['br_num_year'] = bg['br_num'].astype(str) + "_" + bg['txn_end'].str.slice(start=2, stop=4)
# # bg['cp_br_num'] = bg['cp_br_num'].astype('int16')
# bg['cp_br_num'] = bg['cp_br_num'].astype(str)
# bg['cp_br_num_year'] = bg['cp_br_num'].astype(str) + "_" + bg['txn_end'].str.slice(start=2, stop=4)
# bg


,br_num,txn_start,txn_end,data_typ,ser_num,dur_mon,cp_br_num,cp_nm,prop,sup_amt,tax_amt,unknown,br_num_year,cp_br_num_year
0,5038109168,20180101,20180331,5,1,3,5058120418,(주)금창,0.14,6900000.0,690000.0,NaN,5038109168_18,5058120418_18
1,5038109168,20180101,20180331,5,2,3,3128548961,(주)성우홀딩스,0.20,9561500.0,956150.0,NaN,5038109168_18,3128548961_18
2,5038109168,20180101,20180331,5,3,3,5058129326,경동정공(주),0.05,2340704.0,234069.0,NaN,5038109168_18,5058129326_18
3,5038109168,20180101,20180331,5,4,3,1248533444,기아자동차(주),5.74,276380650.0,27638063.0,NaN,5038109168_18,1248533444_18
4,5038109168,20180101,20180331,5,5,3,1338500011,기아자동차(주),1.35,64922766.0,6492275.0,NaN,5038109168_18,1338500011_18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113462472,1232495072,20190701,20191231,8,4,6,1058800491,주식회사 청년들,0.02,100000.0,0.0,NaN,1232495072_19,1058800491_19
113462473,1232495072,20190701,20191231,8,5,6,3128209521,한국가스안전공사가스안전교육원,0.07,289000.0,0.0,NaN,1232495072_19,3128209521_19
113462474,1232495072,20190701,20191231,8,6,6,1398201832,한국양봉농업협동조합,2.79,12387500.0,0.0,NaN,1232495072_19,1398201832_19
113462475,1232495072,20190701,20191231,8,7,6,8684500412,해랑약업사,1.30,5770400.0,0.0,NaN,1232495072_19,8684500412_19


In [39]:
# # ab에 없는 기업 bg에서 날리기
# ab['br_num_year'] = ab['br_num_year'].astype(str)
# bg['br_num_year'] = bg['br_num_year'].astype(str)
# bg['cp_br_num_year'] = bg['cp_br_num_year'].astype(str)
# abli = list(ab['br_num_year'].unique())
# bg = bg[(bg['br_num_year'].isin(abli))&(bg['cp_br_num_year'].isin(abli))]

# # bg에 없는 기업 ab에서 날리기
# bgli = list(set(list(bg['br_num_year'])+list(bg['cp_br_num_year'])))
# # 거래 존재 기업 노드 ab_gr
# ab_gr = ab[ab['br_num_year'].isin(bgli)]
# ab_gr['year'].value_counts()

year
18    70740
19    68756
20    59767
21    50916
22    31685
Name: count, dtype: int64

In [32]:
ab.to_csv('./data/apollo/240826_ab_label만들기 전.csv', index=False)

In [87]:
# 내년 매출액증가율 가져오기

ab_label = ab.copy()

ab_label['br_num'] = ab_label['br_num_year'].str.slice(start=0, stop=10)
ab_label['year'] = ab_label['year'].astype('Int64')
ab_label.sort_values(by=['br_num', 'year'], ascending=[True, True], inplace=True)
groups = ab_label.groupby('br_num')
ab_label['year_diff'] = groups['year'].shift(-1)

ab_label['year_diff'] = ab_label['year_diff'] - ab_label['year']

ab_label.sort_values(by=['br_num', 'year'], ascending=[True, True], inplace=True)

groups_3 = ab_label.groupby('br_num')
ab_label['내년매출액증가율'] = groups_3['매출액증가율'].shift(-1)
ab_label['내년매출액'] = groups_3['매출액'].shift(-1)
ab_label['직접계산매출액증가율'] = (ab_label['내년매출액']-ab_label['매출액']) / ab_label['매출액'] * 100

ab_label = ab_label[ab_label['year_diff']==1]

# ab_label.drop(columns=['br_num','year','year_diff'],inplace=True)

ab_label['diff'] = ab_label['내년매출액증가율'] - ab_label['직접계산매출액증가율']
ab_label.sort_values(by='diff', ascending=False, inplace=True)
ab_label

C:\Users\user\AppData\Local\Temp\ipykernel_6192\562219113.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ab_label['diff'] = ab_label['내년매출액증가율'] - ab_label['직접계산매출액증가율']
C:\Users\user\AppData\Local\Temp\ipykernel_6192\562219113.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ab_label.sort_values(by='diff', ascending=False, inplace=True)


,br_num_year,매출액,인건비,감가상각율,금융비용총부채,금융비용총비용비율,당좌비율,매출액증가율,매출원가율,매출채권회전율,...,재고자산회전율1,총자본회전율,총자산증가율,year,br_num,year_diff,내년매출액증가율,내년매출액,직접계산매출액증가율,diff
721051,1308626678_11,1952958.0,101978.0,36.50,0.23,0.05,485.27,-8.39,67.05,6.52,...,117.41,1.67,7.52,11,1308626678,1,-25.58,1450988.0,-25.703062,0.123062
670761,1138646567_11,800.0,18799.0,0.00,1.66,12.23,5.84,NaN,80.13,NaN,...,0.00,0.00,NaN,11,1138646567,1,73497.13,588777.0,73497.125000,0.005000
769721,5138142091_18,4000.0,46741.0,99.32,0.00,0.00,570.06,-98.46,0.00,0.55,...,NaN,0.01,-21.46,18,5138142091,1,85007.03,3404281.0,85007.025000,0.005000
740602,1298209841_17,20000.0,825887.0,1.87,3.80,45.95,5.86,NaN,1.79,NaN,...,0.08,0.00,7.41,17,1298209841,1,23538.34,4727667.0,23538.335000,0.005000
587495,5088127462_11,20000.0,996.0,NaN,0.00,0.00,15645.30,-68.76,92.00,NaN,...,NaN,0.10,3.62,11,5088127462,1,14534.68,2926935.0,14534.675000,0.005000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832261,8978601294_19,NaN,13.0,NaN,0.00,0.00,2.09,NaN,NaN,NaN,...,NaN,0.00,NaN,19,8978601294,1,NaN,NaN,NaN,NaN
832262,8978601294_20,NaN,8711.0,0.00,5.01,56.96,1.30,NaN,NaN,NaN,...,NaN,0.00,886.09,20,8978601294,1,NaN,16000.0,NaN,NaN
841752,8978801925_20,NaN,28746.0,NaN,1.66,68.43,3.72,NaN,NaN,NaN,...,0.00,0.00,NaN,20,8978801925,1,NaN,8252881.0,NaN,NaN
833589,8998701720_19,NaN,2102.0,NaN,0.33,14.16,106.01,NaN,NaN,NaN,...,NaN,0.00,NaN,19,8998701720,1,NaN,NaN,NaN,NaN


In [91]:
ab_label.drop(columns=['br_num','year_diff','내년매출액','직접계산매출액증가율','diff'], inplace=True)
ab_label.head()

C:\Users\user\AppData\Local\Temp\ipykernel_6192\338760686.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ab_label.drop(columns=['br_num','year_diff','내년매출액','직접계산매출액증가율','diff'], inplace=True)


,br_num_year,매출액,인건비,감가상각율,금융비용총부채,금융비용총비용비율,당좌비율,매출액증가율,매출원가율,매출채권회전율,...,영업이익증가율,유형자산증가율,유형자산회전율,자기자본증가율,자본금회전율,재고자산회전율1,총자본회전율,총자산증가율,year,내년매출액증가율
721051,1308626678_11,1952958.0,101978.0,36.50,0.23,0.05,485.27,-8.39,67.05,6.52,...,-24.83,61.10,32.54,62.37,39.06,117.41,1.67,7.52,11,-25.58
670761,1138646567_11,800.0,18799.0,0.00,1.66,12.23,5.84,NaN,80.13,NaN,...,NaN,NaN,0.02,NaN,0.02,0.00,0.00,NaN,11,73497.13
769721,5138142091_18,4000.0,46741.0,99.32,0.00,0.00,570.06,-98.46,0.00,0.55,...,NaN,-99.32,6.78,-30.47,0.08,NaN,0.01,-21.46,18,85007.03
740602,1298209841_17,20000.0,825887.0,1.87,3.80,45.95,5.86,NaN,1.79,NaN,...,NaN,55.98,0.00,NaN,0.03,0.08,0.00,7.41,17,23538.34
587495,5088127462_11,20000.0,996.0,NaN,0.00,0.00,15645.30,-68.76,92.00,NaN,...,NaN,NaN,NaN,3.19,0.10,NaN,0.10,3.62,11,14534.68


In [92]:
ab_label.drop(columns='year', inplace=True)
ab_label.rename(columns={'내년매출액증가율':'label'}, inplace=True)
ab_label.head()

C:\Users\user\AppData\Local\Temp\ipykernel_6192\4057047477.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ab_label.drop(columns='year', inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_6192\4057047477.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ab_label.rename(columns={'내년매출액증가율':'label'}, inplace=True)


,br_num_year,매출액,인건비,감가상각율,금융비용총부채,금융비용총비용비율,당좌비율,매출액증가율,매출원가율,매출채권회전율,...,수지비율,영업이익증가율,유형자산증가율,유형자산회전율,자기자본증가율,자본금회전율,재고자산회전율1,총자본회전율,총자산증가율,label
721051,1308626678_11,1952958.0,101978.0,36.50,0.23,0.05,485.27,-8.39,67.05,6.52,...,79.98,-24.83,61.10,32.54,62.37,39.06,117.41,1.67,7.52,-25.58
670761,1138646567_11,800.0,18799.0,0.00,1.66,12.23,5.84,NaN,80.13,NaN,...,6030.28,NaN,NaN,0.02,NaN,0.02,0.00,0.00,NaN,73497.13
769721,5138142091_18,4000.0,46741.0,99.32,0.00,0.00,570.06,-98.46,0.00,0.55,...,660.97,NaN,-99.32,6.78,-30.47,0.08,NaN,0.01,-21.46,85007.03
740602,1298209841_17,20000.0,825887.0,1.87,3.80,45.95,5.86,NaN,1.79,NaN,...,14853.03,NaN,55.98,0.00,NaN,0.03,0.08,0.00,7.41,23538.34
587495,5088127462_11,20000.0,996.0,NaN,0.00,0.00,15645.30,-68.76,92.00,NaN,...,77.92,NaN,NaN,NaN,3.19,0.10,NaN,0.10,3.62,14534.68


In [93]:
ab_label.to_csv('./data/apollo/240816_전체년도ab.csv', index=False)

In [ ]:
import pandas as pd
ab = pd.read_csv('./data/apollo/240816_전체년도ab.csv')
ab

In [ ]:
ab['year'] = ab['br_num_year'].str.slice(start=11, stop=13)
ab['br_num'] = ab['br_num_year'].str.slice(start=0, stop=10)

In [ ]:
ab.columns

In [ ]:
ab.describe()

In [ ]:
ignore_cols = ['br_num_year','매출액','인건비','year','br_num']
cols = [col for col in ab.columns if col not in ignore_cols]
ratio_outlier_abli = []

for col in cols:
    outlier_abli = list(ab[(ab[col]>=100000) | (ab[col]<= -1000)]['br_num'].unique())
    ratio_outlier_abli.append(outlier_abli)
    print(f'{col} 이상치 해당 기업 수 : {len(outlier_abli)}')

mce_outlier_abli = ab[abs(ab['매출액'])<=1000]['br_num'].unique()
print(f'매출액 이상치 해당 기업 수 : {len(mce_outlier_abli)}')

ratio_outlier_abli.append(mce_outlier_abli)

# 2차원 리스트를 1차원 리스트로 변환
flattened_list = [item for sublist in ratio_outlier_abli for item in sublist]

# 중복된 값을 제거하여 1차원 리스트로 변환
unique_list = list(set(flattened_list))

allabcnt = len(list(ab['br_num'].unique()))
print(f'전체 기업 수 : {allabcnt}')
print(f'이상치 해당 기업 전체 수 : {len(unique_list)}')

In [ ]:
ab_outlier = ab[~ab['br_num'].isin(unique_list)]
len(ab), len(ab_outlier)

In [ ]:
len(ab) - len(ab_outlier)

In [ ]:
# 결측치 평균 대체

ab_outlier.isnull().sum()/len(ab_outlier)

In [ ]:
# 결측치 평균대체

# 숫자형 열만 선택하여 결측치 대체 수행
import numpy as np
numeric_cols = ab_outlier.select_dtypes(include=[np.number])

# 각 열의 평균값으로 결측치를 대체
ab_outlier[numeric_cols.columns] = numeric_cols.apply(lambda x: x.fillna(x.mean()), axis=0)
ab_outlier.isnull().sum()/len(ab_outlier)

In [ ]:
# lag-1 (매출액증가율_lag1) 변수 가져오기

ab_lag = ab_outlier.copy()

ab_lag['br_num'] = ab_lag['br_num_year'].str.slice(start=0, stop=10)
ab_lag['year'] = ab_lag['year'].astype('Int64')
ab_lag.sort_values(by=['br_num', 'year'], ascending=[True, True], inplace=True)
groups = ab_lag.groupby('br_num')
ab_lag['year_diff'] = groups['year'].shift(1)

ab_lag['year_diff'] = ab_lag['year_diff'] - ab_lag['year']

ab_lag.sort_values(by=['br_num', 'year'], ascending=[True, True], inplace=True)

groups_3 = ab_lag.groupby('br_num')
ab_lag['매출액증가율_lag1'] = groups_3['매출액증가율'].shift(1)

ab_lag = ab_lag[ab_lag['year_diff']==-1]

# ab_label.drop(columns=['br_num','year','year_diff'],inplace=True)

# ab_lag

# lag-2 (매출액증가율_lag1) 변수 가져오기
ab_lag2 = ab_lag.copy()

ab_lag2.sort_values(by=['br_num', 'year'], ascending=[True, True], inplace=True)
groups = ab_lag2.groupby('br_num')
ab_lag2['year_diff'] = groups['year'].shift(2)

ab_lag2['year_diff'] = ab_lag2['year_diff'] - ab_lag2['year']

ab_lag2.sort_values(by=['br_num', 'year'], ascending=[True, True], inplace=True)

groups_3 = ab_lag2.groupby('br_num')
ab_lag2['매출액증가율_lag2'] = groups_3['매출액증가율'].shift(2)

ab_lag2 = ab_lag2[ab_lag2['year_diff']==-2]
ab_lag2

In [ ]:
ab_label = ab_lag2.copy()
ab_label['label_1'] = ab_label['label'].apply(lambda x : 1 if x>=10 else 0)
ab_label['label_2'] = ab_label['label'].apply(lambda x : 1 if x>=20 else 0)

ab_label

In [ ]:
ab_label['label_1'].value_counts()/len(ab_label), ab_label['label_2'].value_counts()/len(ab_label)

In [ ]:
ab_label.drop(columns=['br_num','year','year_diff'], inplace=True)
ab_label

In [ ]:
ab_label.describe()

In [ ]:
# 매출액, 인건비 로그변환

cols = ['매출액', '인건비']
for col in cols:
    if ab_label[col].min() < 1:
        eps = abs(ab_label[col].min())+1
        ab_label[col] = np.log(ab_label[col]+eps)
ab_label.describe()

In [ ]:
ab_label.to_csv('./data/apollo/240824_전체년도ab_결측치이상치처리완료_v3.csv', index=False)

# ab 전처리 완료 (240824_전체년도ab_결측치이상치처리완료_v3.csv)

1. GAAP 매핑
2. 이상치 제거 ex) 증가율 억단위
3. 항목명 특수문자, 연속공백, 공백 제거
4. co_code기준 em01의 br_num, fin_year, cl_code와 매핑
5. fin_year와 ref_date 같은 달에 해당하는 녀석만 남기기 (결산월 맞추기)
6. cl_code N인 녀석만 남기기 (폐쇄 기업 아닌 경우만)
7. 한 기업 한 년도를 하나의 row로 만들기
8. 업체코드, 기준일자, 항목명 중복시 제거 (첫번째 row 남기기)
9. 풀 데이터 컬럼만 남기기 (업체코드 제외하고 23개 독립변수, 1개 종속변수)
10. ab01, ab02 매핑
11. 내년매출액증가율 (LABEL) 만들기

- 이상치 제거 먼저
    - 비율 데이터 1000배 이상 or -10배 이하 or 매출액이 백만원 이하인 경우에 해당하는 기업 삭제
- 결측값 --> 평균으로 대체
- 매출액, 인건비 로그변환(전체 년도에 대해서 일관되게 변환)
- lag 변수 2개 추출 (매출액증가율)